In [1]:
# default_exp core

# Core Functions
> Python library to access online data from the Joshua Project via APIv2

[Joshua Project](https://joshuaproject.net/) via [APIv2](https://joshuaproject.net/resources/datasets)

In [2]:
# export
import urllib3
from urllib3.util.url import Url
import certifi
import json
from functools import partial
from datetime import date

In [3]:
#hide
from nbdev.showdoc import *

## URL Construction Functions

In [4]:
# hide
api_key='w3NOS49TW7fg' #'get_your_key'

In [5]:
# export
def url_path_query(path:str,cntry:str=None,pgid:str=None,extra=None,month=None,day=None,api_key='w3NOS49TW7fg')->str:
    """Builds and returns a string url to query `path` for `cntry` 
    with optional `pgid`,`extra`,`month` and `day`."""
    query = ''
    if cntry is not None: query = query+'ROG3='+cntry
    if pgid  is not None: query = query+'&PeopleID3='+pgid
    if extra is not None: query = query+'&'+extra
    if 'upgotd' in path:
        today = date.today()
        if day is None:   day   = today.strftime("%-d")
        if month is None: month = today.strftime("%-m")
        query = query+'&LRofTheDayMonth='+str(month) 
        query = query+'&LRofTheDayDay='  +str(day)
    query = query+'&api_key='+api_key
    return Url(scheme='https', host='joshuaproject.net', path=path, query=query).url

In [6]:
url_path_query('/api/v2/people_groups')

'https://joshuaproject.net/api/v2/people_groups?&api_key=w3NOS49TW7fg'

In [7]:
show_doc(url_path_query)

<h4 id="url_path_query" class="doc_header"><code>url_path_query</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>url_path_query</code>(**`path`**:`str`, **`cntry`**:`str`=*`None`*, **`pgid`**:`str`=*`None`*, **`extra`**=*`None`*, **`month`**=*`None`*, **`day`**=*`None`*, **`api_key`**=*`'w3NOS49TW7fg'`*)

Builds and returns a string url to query `path` for `cntry` 
with optional `pgid`,`extra`,`month` and `day`.

In [8]:
# export
url_pgs_cntry  = partial(url_path_query,'/api/v2/people_groups')
url_pgs_cntry.__doc__ = """Get all people groups in a specific country."""

url_pg_cntry   = partial(url_pgs_cntry)
url_pg_cntry.__doc__ = """Get a specific people group in a specific country."""

url_upgs_cntry = partial(url_pgs_cntry,extra='LeastReached=Y')
url_upgs_cntry.__doc__ = """Get all unreached people groups in a specific country."""

url_upgotd     = partial(url_path_query,'/api/v2/upgotd')
url_upgotd.__doc__ = """Get data for the UPG of the day."""

In [9]:
url_upgotd()

'https://joshuaproject.net/api/v2/upgotd?&LRofTheDayMonth=1&LRofTheDayDay=24&api_key=w3NOS49TW7fg'

In [10]:
url_upgotd(day=10,month=2)

'https://joshuaproject.net/api/v2/upgotd?&LRofTheDayMonth=2&LRofTheDayDay=10&api_key=w3NOS49TW7fg'

In [11]:
url_upgs_cntry('IN')

'https://joshuaproject.net/api/v2/people_groups?ROG3=IN&LeastReached=Y&api_key=w3NOS49TW7fg'

In [12]:
# assert  url_pgs_cntry('EN') == 'http://joshuaproject.net/api/v2/people_groups?ROG3=EN&api_key=w3NOS49TW7fg'
# assert url_upgs_cntry('EN') == 'http://joshuaproject.net/api/v2/people_groups?ROG3=EN&LeastReached=Y&api_key=w3NOS49TW7fg'

## Testing

In [13]:
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())

In [14]:
url = url_pgs_cntry('RS')
url

'https://joshuaproject.net/api/v2/people_groups?ROG3=RS&api_key=w3NOS49TW7fg'

In [15]:
resp = http.request('GET',url)

In [16]:
if resp.status != 200:
    resp_j = json.loads(resp.data)
    s = resp_j['status']['status_code']
    m = resp_j['status']['message']
    print(f'{s}: {m}')


In [17]:
url = url_pg_cntry('IN','16152')
url

'https://joshuaproject.net/api/v2/people_groups?ROG3=IN&PeopleID3=16152&api_key=w3NOS49TW7fg'

In [18]:
resp_j = json.loads(resp.data)

In [19]:
# hide
print(json.dumps(resp_j['data'], sort_keys=False, indent=2))

[
  {
    "PeopleID3ROG3": "10125RS",
    "ROG3": "RS",
    "Ctry": "Russia",
    "CtryShort": "Russia",
    "CtryLiteracyRate": 100,
    "CtryLiteracySource": "Ethnologue 2018",
    "PeopleID3": 10125,
    "ROP3": 100011,
    "PeopNameInCountry": "Abaza",
    "PeopNameAcrossCountries": "Abaza",
    "Population": 45000,
    "ROG2": "ASI",
    "Continent": "Asia",
    "RegionCode": 9,
    "RegionName": "Europe, Eastern and Eurasia",
    "10_40Window": "N",
    "LocationInCountry": "Karachayevo-Cherkesiya and Stavropol\u2019skiy Kray.",
    "PeopleID1": 12,
    "AffinityBloc": "Eurasian Peoples",
    "PeopleID2": 157,
    "PeopleCluster": "Caucasus",
    "JPScalePC": 2,
    "PopulationPGAC": 73000,
    "JPScalePGAC": 1,
    "JPScale": 1,
    "JPScaleTemp": null,
    "LeastReached": "Y",
    "Frontier": "Y",
    "WorkersNeeded": 1,
    "Subgroups": null,
    "ROL3": "abq",
    "PrimaryLanguageName": "Abaza",
    "ROL4": 0,
    "PrimaryLanguageDialect": null,
    "Written": "Y",
    "Bible

In [20]:
resp_j['meta']['pagination']

{'total_count': 180, 'total_pages': 2, 'current_page': 1, 'limit': 100}

In [21]:
# TODO: Add procedding for multi-page results
# for i in range(resp_j['meta']['pagination']['total_count']):
#     print(resp_j['data'][i]['PeopNameInCountry'])

In [22]:
resp_j['data'][0]

{'PeopleID3ROG3': '10125RS',
 'ROG3': 'RS',
 'Ctry': 'Russia',
 'CtryShort': 'Russia',
 'CtryLiteracyRate': 100,
 'CtryLiteracySource': 'Ethnologue 2018',
 'PeopleID3': 10125,
 'ROP3': 100011,
 'PeopNameInCountry': 'Abaza',
 'PeopNameAcrossCountries': 'Abaza',
 'Population': 45000,
 'ROG2': 'ASI',
 'Continent': 'Asia',
 'RegionCode': 9,
 'RegionName': 'Europe, Eastern and Eurasia',
 '10_40Window': 'N',
 'LocationInCountry': 'Karachayevo-Cherkesiya and Stavropol’skiy Kray.',
 'PeopleID1': 12,
 'AffinityBloc': 'Eurasian Peoples',
 'PeopleID2': 157,
 'PeopleCluster': 'Caucasus',
 'JPScalePC': 2,
 'PopulationPGAC': 73000,
 'JPScalePGAC': 1,
 'JPScale': 1,
 'JPScaleTemp': None,
 'LeastReached': 'Y',
 'Frontier': 'Y',
 'WorkersNeeded': 1,
 'Subgroups': None,
 'ROL3': 'abq',
 'PrimaryLanguageName': 'Abaza',
 'ROL4': 0,
 'PrimaryLanguageDialect': None,
 'Written': 'Y',
 'BibleStatus': 0,
 'NumberLanguagesSpoken': 1,
 'RLG3': 6,
 'PrimaryReligion': 'Islam',
 'RLG4': None,
 'ReligionSubdivision'

In [23]:
import pandas as pd

In [24]:
cc = pd.read_csv('data/country_codes.csv',skiprows=3,names=['Country_name','FIPS','ISO'])

In [25]:
cc.head().to_dict(orient='records')

[{'Country_name': 'Afghanistan', 'FIPS': 'AF', 'ISO': 'AF'},
 {'Country_name': 'Aland Islands', 'FIPS': nan, 'ISO': 'AX'},
 {'Country_name': 'Albania', 'FIPS': 'AL', 'ISO': 'AL'},
 {'Country_name': 'Algeria', 'FIPS': 'AG', 'ISO': 'DZ'},
 {'Country_name': 'American Samoa', 'FIPS': 'AQ', 'ISO': 'AS'}]

In [26]:
cci = cc.set_index('ISO')

In [27]:
cci.drop('Country_name',axis=1).to_dict(orient='records')

[{'FIPS': 'AF'},
 {'FIPS': nan},
 {'FIPS': 'AL'},
 {'FIPS': 'AG'},
 {'FIPS': 'AQ'},
 {'FIPS': 'AN'},
 {'FIPS': 'AO'},
 {'FIPS': 'AV'},
 {'FIPS': 'AY'},
 {'FIPS': 'AC'},
 {'FIPS': 'AR'},
 {'FIPS': 'AM'},
 {'FIPS': 'AA'},
 {'FIPS': 'AS'},
 {'FIPS': 'AU'},
 {'FIPS': 'AJ'},
 {'FIPS': 'BF'},
 {'FIPS': 'BA'},
 {'FIPS': 'BG'},
 {'FIPS': 'BB'},
 {'FIPS': 'BO'},
 {'FIPS': 'BE'},
 {'FIPS': 'BH'},
 {'FIPS': 'BN'},
 {'FIPS': 'BD'},
 {'FIPS': 'BT'},
 {'FIPS': 'BL'},
 {'FIPS': nan},
 {'FIPS': 'BK'},
 {'FIPS': 'BC'},
 {'FIPS': 'BV'},
 {'FIPS': 'BR'},
 {'FIPS': 'IO'},
 {'FIPS': 'BX'},
 {'FIPS': 'BU'},
 {'FIPS': 'UV'},
 {'FIPS': 'BY'},
 {'FIPS': 'CV'},
 {'FIPS': 'CB'},
 {'FIPS': 'CM'},
 {'FIPS': 'CA'},
 {'FIPS': 'CJ'},
 {'FIPS': 'CT'},
 {'FIPS': 'CD'},
 {'FIPS': 'CI'},
 {'FIPS': 'CH'},
 {'FIPS': 'KT'},
 {'FIPS': 'CK'},
 {'FIPS': 'CO'},
 {'FIPS': 'CN'},
 {'FIPS': 'CF'},
 {'FIPS': 'CG'},
 {'FIPS': 'CW'},
 {'FIPS': 'CS'},
 {'FIPS': 'IV'},
 {'FIPS': 'HR'},
 {'FIPS': 'CU'},
 {'FIPS': 'UC'},
 {'FIPS': 'CY'},

In [28]:
cci.to_dict(orient='dict')['Country_name']

{'AF': 'Afghanistan',
 'AX': 'Aland Islands',
 'AL': 'Albania',
 'DZ': 'Algeria',
 'AS': 'American Samoa',
 'AD': 'Andorra',
 'AO': 'Angola',
 'AI': 'Anguilla',
 'AQ': 'Antarctica',
 'AG': 'Antigua and Barbuda',
 'AR': 'Argentina',
 'AM': 'Armenia',
 'AW': 'Aruba',
 'AU': 'Australia',
 'AT': 'Austria',
 'AZ': 'Azerbaijan',
 'BS': 'Bahamas',
 'BH': 'Bahrain',
 'BD': 'Bangladesh',
 'BB': 'Barbados',
 'BY': 'Belarus',
 'BE': 'Belgium',
 'BZ': 'Belize',
 'BJ': 'Benin',
 'BM': 'Bermuda',
 'BT': 'Bhutan',
 'BO': 'Bolivia, Plurinational State Of',
 'BQ': 'Bonaire, Sint Eustatius and Saba',
 'BA': 'Bosnia and Herzegovina',
 'BW': 'Botswana',
 'BV': 'Bouvet Island',
 'BR': 'Brazil',
 'IO': 'British Indian Ocean Territory',
 'BN': 'Brunei Darussalam',
 'BG': 'Bulgaria',
 'BF': 'Burkina Faso',
 'BI': 'Burundi',
 'CV': 'Cabo Verde',
 'KH': 'Cambodia',
 'CM': 'Cameroon',
 'CA': 'Canada',
 'KY': 'Cayman Islands',
 'CF': 'Central African Republic',
 'TD': 'Chad',
 'CL': 'Chile',
 'CN': 'China',
 'CX'

### Convert JSON into a Pandas DataFrame

In [29]:
pd.DataFrame(resp_j['data']).head()

,PeopleID3ROG3,ROG3,Ctry,CtryShort,CtryLiteracyRate,CtryLiteracySource,PeopleID3,ROP3,PeopNameInCountry,PeopNameAcrossCountries,...,MapCreditURL,MapCopyright,GSEC,JF,AudioRecordings,NTOnline,GospelRadio,PEID,Longitude,Latitude
0,10125RS,RS,Russia,Russia,100,Ethnologue 2018,10125,100011,Abaza,Abaza,...,None,N,1,N,N,,N,48739.0,41.889520,44.221680
1,10130RS,RS,Russia,Russia,100,Ethnologue 2018,10130,100018,Abkhaz,Abkhaz,...,http://en.wikipedia.org/wiki/File:Caucasus_eth...,N,,Y,N,,N,NaN,39.966281,43.412621
2,10161RS,RS,Russia,Russia,100,Ethnologue 2018,10161,100079,"Adyghe, West Circassian",Adyghe,...,None,N,3,Y,Y,,N,24434.0,40.113917,44.594719
3,10184RS,RS,Russia,Russia,100,Ethnologue 2018,10184,100126,Aghul,Aghul,...,None,Y,1,N,Y,,N,24403.0,47.554020,41.930850
4,10224RS,RS,Russia,Russia,100,Ethnologue 2018,10224,100187,Akhvakh,Akhvakh,...,None,N,0,N,N,,N,24405.0,46.326160,42.384760


In [30]:
#hide
json_str = json.dumps(resp_j['data'])

All people groups in a specific country

http://joshuaproject.net/api/v2/people_groups?ROG3=YM&api_key=aWye9lV20QtF

All unreached people groups in a specific country

https://joshuaproject.net/api/v2/people_groups?ROG3=YM&LeastReached=Y&api_key=aWye9lV20QtF

A people group in a specific country

http://joshuaproject.net/api/v2/people_groups?ROG3=AF&PeopleID3=19409&api_key=aWye9lV20QtF

All countries a specific people group lives in

http://joshuaproject.net/api/v2/people_groups?PeopleID3=10294&api_key=aWye9lV20QtF

**Country summary data**

http://joshuaproject.net/api/v2/countries?ROG3=AF&api_key=aWye9lV20QtF

**Language summary info**

http://joshuaproject.net/api/v2/languages?ROL3=hau&api_key=aWye9lV20QtF

**Unreached of the Day** (basic data October 16th)

https://joshuaproject.net/api/v2/upgotd?api_key=aWye9lV20QtF&ROL3Profile=eng&LRofTheDayMonth=10&LRofTheDayDay=16

## Export -

In [31]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
